This is a Neural Network to predict commitment of students with online courses.

In [113]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt

In [118]:
data = pd.read_csv("out.csv")
data.describe()

,engagement,anger,contempt,disgust,fear,happiness,neutral,sadness,surprise
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,0.505380,7.447691,7.527219,7.468637,7.491103,7.474113,7.477786,7.450546,7.548911
std,0.292393,4.342329,4.344747,4.327625,4.325684,4.327359,4.313112,4.343131,4.311707
min,0.000000,0.000365,0.001694,0.001612,0.000662,0.002418,0.000616,0.000678,0.000552
25%,0.300000,3.688172,3.741956,3.720413,3.747869,3.740014,3.770971,3.693497,3.828738
50%,0.500000,7.433271,7.596299,7.429996,7.552279,7.411314,7.525319,7.423179,7.546941
75%,0.800000,11.228078,11.292224,11.206452,11.198329,11.250509,11.156155,11.212856,11.273236
max,1.000000,14.998577,14.997546,14.998057,14.998981,14.999357,14.998968,14.999237,14.999866


In [193]:
quant_features = ['engagement', 'anger', 'contempt', 'disgust', 'fear', 'happiness','neutral',
                  'sadness','surprise']
# Store scalings in a dictionary so we can convert back later
scaled_features = {}
for each in quant_features:
    mean, std = data[each].mean(), data[each].std()
    scaled_features[each] = [mean, std]
    data.loc[:, each] = (data[each] - mean)/std

Aqui, vamos dividir os dados para treinamento, teste e validação da rede neural.

In [194]:
train, validate, test = np.split(data.sample(frac=1), [int(.6*len(data)), int(.8*len(data))])

In [195]:
#dado alvo 

target_field = ['engagement']
features, target = data.drop(target_field, axis=1), data[target_field]
test_features, test_target = test.drop(target_field, axis=1), test[target_field]

In [196]:
#Dados para treinamento
train_features, train_target = features[:8000], target[:0]
val_features, val_target = features[-8000:], target[0:]

In [204]:
scaled_features = {}
for each in quant_features:
    mean, std = data_train[each].mean(), data_train[each].std()
    scaled_features[each] = [mean, std]
    data_train.loc[:, each] = (data_train[each] - mean)/std
scaled_features

{'engagement': [6.631732200427602e-17, 1.0],
 'anger': [-1.1842378929335004e-17, 1.0],
 'contempt': [3.0790185216271007e-17, 1.0],
 'disgust': [2.2500519965736505e-17, 1.0],
 'fear': [-5.3290705182007515e-18, 1.0],
 'happiness': [-2.3684757858670005e-18, 1.0],
 'neutral': [3.789561257387201e-17, 1.0],
 'sadness': [2.3092638912203255e-17, 1.0],
 'surprise': [-4.500103993147301e-17, 1.0]}

In [235]:
class RedeNeural(object):
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        #definir o numero de hidden nodes
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes
        #pesos estatísticos
        self.weights_input_to_hidden = np.random.normal(0.0, self.hidden_nodes**-0.5, 
                                       (self.hidden_nodes, self.input_nodes))

        self.weights_hidden_to_output = np.random.normal(0.0, self.output_nodes**-0.5, 
                                       (self.output_nodes, self.hidden_nodes))
        self.lr = learning_rate
        
        #A distrubuição aqui é sigmoide
        self.activation_function = lambda x: (1 / (1 + np.exp(-x)))
        self.activation_function_prime = lambda x: (x * ( 1 - x))
        
        self.output_activation_function = lambda x: (x)
        self.output_activation_function_prime = lambda x: (1)
        
    def train(self, inputs, targets_list):
        inputs = np.array(inputs, ndmin=2).T
        targets = np.array(targets, ndmin=2).T
        #convertendo em vetores 2D
        
        
        hidden_inputs = np.dot(self.weights_input_to_hidden, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        final_inputs = np.dot(self.weights_hidden_to_output, hidden_outputs)
        final_outputs = self.output_activation_function(final_inputs)
        

        error = targets - final_outputs
        
        output_errors = error * self.output_activation_function_prime(final_inputs)
        #erro propagado
        hidden_errors = np.dot(output_errors.T, self.weights_hidden_to_output).T
        hidden_grad = self.activation_function_prime(hidden_outputs)
        
 
        self.weights_hidden_to_output += self.lr * np.dot(output_errors, hidden_outputs.T)
        self.weights_input_to_hidden += self.lr * np.dot((hidden_errors * hidden_grad), inputs.T)
 
        
    def run(self, inputs):
        inputs = np.array(inputs, ndmin=2).T
        
       
        hidden_inputs = np.dot(self.weights_input_to_hidden, inputs)
        hidden_outputs = self.activation_function(hidden_inputs,inputs)
        
        final_inputs = np.dot(self.weights_hidden_to_output, hidden_outputs) 
        final_outputs = self.output_activation_function(final_inputs,outputs)
        
        return final_outputs
    

In [206]:
def EQMedio(y, Y):
    return np.mean((y-Y)**2)

In [229]:
all_data = pd.concat((train,test))
for column in all_data.select_dtypes(include=[np.object]).columns:
    print(column, all_data[column].unique())

In [230]:
all_data = pd.concat((train,test))
for column in all_data.select_dtypes(include=[np.object]).columns:
    print(column, all_data[column].unique())

In [231]:
x = np.arange(4)

In [252]:
import sys

epochs = 100
learning_rate = 0.05
hidden_nodes = 20
output_nodes = 1

N_i = train.shape[1]
network = RedeNeural(N_i, hidden_nodes, output_nodes, learning_rate)

losses = {'train':[], 'validation':[]}
for e in range(epochs):
    batch = np.random.choice(train_features.index, size=128)
    
    foo = train_targets.ix[batch]['engagement'].values
    print(foo)
    
    for record, target in zip(train_features.index[batch].values, foo):network.train(record, target)
    
    foo2 = train_targets.ix[batch]['engagement'].values
    train_loss = EQMedio(network.run(train_features), train_targets['engagement'].values)
    val_loss = EQMedio(network.run(val_features), val_targets['engagement'].values)
    sys.stdout.write("\rProgress: " + str(100 * e/float(epochs))[:4] \
                     + "% ... Training loss: " + str(train_loss)[:5] \
                     + " ... Validation loss: " + str(val_loss)[:5])
    
    losses['train'].append(train_loss)
    losses['validation'].append(val_loss)


c:\users\ivan\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  from ipykernel import kernelapp as app


In [165]:
y = data.as_matrix(columns=data.columns[0:1])
x = data.as_matrix(columns=data.columns[1:])
data.columns[0:]

c:\users\ivan\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
c:\users\ivan\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


Index(['engagement', 'anger', 'contempt', 'disgust', 'fear', 'happiness',
       'neutral', 'sadness', 'surprise'],
      dtype='object')